# Pré processamento dos dados do arquivo classificacao_crimes.csv

In [ ]:
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from unidecode import unidecode
import spacy
import pandas as pd
from collections import Counter
from itertools import chain

nlp = spacy.load("pt_core_news_sm")
PT_STOPWORDS = set(stopwords.words("portuguese"))
STEMMER = RSLPStemmer()

In [ ]:
df_crimes_limpo = pd.read_csv('../data/01_dados_selecionados.csv')

Eliminando expressões regulares do texto

In [37]:
def limpeza_de_texto(texto):
    t = texto.lower()
    t = unidecode(t)
    t = re.sub(r"\s+", ' ', t) #nova linha
    t = re.sub(r"https?://\S+|www\.\S+", ' ', t) #url
    t = re.sub(r"<[^>]+>", ' ', t) #padrões html
    t = re.sub(r"&\w+", ' ', t) #e comercial
    t = re.sub(r"@\w+\b", ' ', t) #menções a @ de rede social
    t = re.sub(r"\b[\w.-]+@[\w.-]+\.\w{2,}\b", ' ', t) #emails
    t = re.sub(r"\d+", ' ', t) #números
    t = re.sub(r"/", ' ', t) #barras
    t = t.translate(str.maketrans('', '', string.punctuation))

    return t

df_crimes_limpo['Texto limpo'] = df_crimes_limpo['Relato'].apply(limpeza_de_texto)
pd.set_option('display.max_colwidth', None) 

df_crimes_limpo.head(3)


,Classificação,Relato,Quantidade de chracteres,Quantidade de palavras,Texto limpo
0,AMEACA,"<P>A&NBSP;COMUNICANTE ACIMA QUALIFICADA&NBSP;COMPARECEU NESSA CENTRAL DE POLICIA DE SAO BRAS PARA INFORMAR QUE NA HORA, DATA E LOCAL ACIMA CITADOS FOI VITIMA DE AMEACA POR PARTE DA NACIONAL JUCELENA GOMES OLIVEIRA, ESTA RESIDIA ATE NOV/20 EM CONJUNTO COM A RELATORA, POREM ESSA MUDOU-SE PARA CAMPINAS/SP ONDE ESTA ATE O PRESENTE MOMENTO, TENDO EXPECTATIVA&NBSP;DE RETORNO PARA BELEM/PA NOS PROXIMOS DIAS AO REGISTRO DESTE B.O.P. A ACUSADA REALIZA CONSTANTES OFENSAS E AMEACAS A INTEGRIDADE FISICA DA MESMA E DE SEUS FAMILIARES, AO REIVINDICAR POSSE DO IMOVEL ONDE RESIDIU ANTERIORMENTE. REGISTRA-SE PARA OS DEVIDOS FINS DE DIREITO. ////////////////////</P>",656,95,a comunicante acima qualificada compareceu nessa central de policia de sao bras para informar que na hora data e local acima citados foi vitima de ameaca por parte da nacional jucelena gomes oliveira esta residia ate nov em conjunto com a relatora porem essa mudouse para campinas sp onde esta ate o presente momento tendo expectativa de retorno para belem pa nos proximos dias ao registro deste bop a acusada realiza constantes ofensas e ameacas a integridade fisica da mesma e de seus familiares ao reivindicar posse do imovel onde residiu anteriormente registrase para os devidos fins de direito
1,AMEACA,"<p>//A comunicante acima qualificada compareceu a esta Seccional Urbana para informar que no dia 21/06/2020, por volta das 12:05 estava fazendo seu alimento no fogo a lenha pos avia terminado seu&nbsp;gás, quando sua vizinha a Sra de suposto Nominal RAIMUNDA residente na PASSAGEM; CABO LEAO E/ RUA TAMBES E SANTA ANA N° 25 ALTOS ela por sua vez&nbsp;se sentiu incomodada com a fumaça e começou e começou a&nbsp;agredir verbalmente a relatora, que informa que não ser a primeira vez que acontece, e por isso veio ate esta Seccional para se fazer um B.O.P, Regista-se para fins de direito///.</p>",595,98,a comunicante acima qualificada compareceu a esta seccional urbana para informar que no dia por volta das estava fazendo seu alimento no fogo a lenha pos avia terminado seu gas quando sua vizinha a sra de suposto nominal raimunda residente na passagem cabo leao e rua tambes e santa ana ndeg altos ela por sua vez se sentiu incomodada com a fumaca e comecou e comecou a agredir verbalmente a relatora que informa que nao ser a primeira vez que acontece e por isso veio ate esta seccional para se fazer um bop registase para fins de direito
2,ESTUPRO DE VULNERAVEL,"<P>A RELATORA ATUALMENTE COM 21 ANOS DE IDADE, VEM A ESTA SECCIONAL COMUNICAR QUE QUANDO TINHA 14 ANOS DE IDADE, FOI ABUSADA SEXUALMENTE POR SEU PADRASTO DE PRE NOME MANOEL VULGO “AGUIA”; QUE RELATA QUE EM 2015 QUANDO TINHA A IDADE DE 14 ANOS, A RELATORA MORAVA COM SUA AVO MATERNA E UM PRIMO SEU O QUAL ERA PORTADOR DE NECESSIDADES ESPECIUAIS, EM UMA OUTRA CASA LOCALIZADA NO BAIRRO DO MARACAJA E SUA GENITORA MORAVA NO BAIRRO DO PANTANAL JUNTAMENTE COM SEU ABUSADOR O NACIONAL MANOEL VULGO “AGUIA”; QUE SUA MAE BIOLOGICA NA EPOCA, IA PARA CASA DA AVO DA RELATORA CUIDAR DO TAL PRIMO ESPECIAL, QUE ERA UMA ESPECIE DE EMPREGO DA GENITORA DA RELATORA, POIS A SUA AVO PAGA UMA CERTA QUANTIA EM DINHEIRO PARA SUA MAE; QUE MANOEL VULGO AGUIA FICAVA SOZINHO NA CASA ONDE MORAVAM NO BAIRRO DO PANTANAL; QUE EM DETERMINADO DIA DO ANO DE 2015, A MAE BIOLOGICA DA RELATORA PEDIU PARA QUE ELA (A VITIMA) FOSSE ATE A CASA ONDE ESTAVA MANOEL VULGO AGUIA PARA LEVAR UM PRATO DE COMIDA PARA ELE, O QUAL ESTAVA SOZINHO; QUE A RELATORA ENTAO FOI, CHEGANDO NO LOCAL ENCONTROU COM MANOEL E ESTE LHE DEU UM DINHEIRO E PEDIU PARA QUE A RELATORA FOSSE COMPRAR UM SUCO PARA ELE, QUE A RELATORA ENTAO FOI COMPRAR; QUE AO RETORNAR PARA A CASA ONDE ESTAVA MANOEL PARA LHE ENTREGAR O SUCO, ESTE JA ESTAVA SOMENTE DE CUECAS; QUE, MANDOU QUE A RELATORA ENTRASSE, E ASSIM QUE ELA ENTROU MANOEL A EMPURROU NO CHAO E MANDOU QUE FICASSE CALADA, TIROU A ROUPA DA RELATORA E PRATICOU SEXO COM A MESM

tokenização, stop words, stemminização, lemmatização

In [38]:
def tokenizar(texto):
    return texto.split()

def remover_stopwords(tokens):
    off_stopwords = []
    for tok in tokens:
        if tok not in PT_STOPWORDS:
            off_stopwords.append(tok)
    return off_stopwords

def stemminizar(tokens):
    stemmins = []
    for tok in tokens:
        stemmins.append(STEMMER.stem(tok))
    return stemmins

def lemmatizar(texto):
    doc = nlp(texto)
    lemmas = []
    for t in doc:
        lemmas.append(t.lemma_)
    return lemmas

df_crimes_limpo['Tokens'] = df_crimes_limpo['Texto limpo'].apply(tokenizar)
df_crimes_limpo['Sem stopwords'] = df_crimes_limpo['Tokens'].apply(remover_stopwords)
df_crimes_limpo['Stem tokens'] = df_crimes_limpo['Sem stopwords'].apply(stemminizar)
df_crimes_limpo['Lemma tokens'] = df_crimes_limpo['Texto limpo'].apply(lemmatizar)

df_crimes_limpo.head(3)

,Classificação,Relato,Quantidade de chracteres,Quantidade de palavras,Texto limpo,Tokens,Sem stopwords,Stem tokens,Lemma tokens
0,AMEACA,"<P>A&NBSP;COMUNICANTE ACIMA QUALIFICADA&NBSP;COMPARECEU NESSA CENTRAL DE POLICIA DE SAO BRAS PARA INFORMAR QUE NA HORA, DATA E LOCAL ACIMA CITADOS FOI VITIMA DE AMEACA POR PARTE DA NACIONAL JUCELENA GOMES OLIVEIRA, ESTA RESIDIA ATE NOV/20 EM CONJUNTO COM A RELATORA, POREM ESSA MUDOU-SE PARA CAMPINAS/SP ONDE ESTA ATE O PRESENTE MOMENTO, TENDO EXPECTATIVA&NBSP;DE RETORNO PARA BELEM/PA NOS PROXIMOS DIAS AO REGISTRO DESTE B.O.P. A ACUSADA REALIZA CONSTANTES OFENSAS E AMEACAS A INTEGRIDADE FISICA DA MESMA E DE SEUS FAMILIARES, AO REIVINDICAR POSSE DO IMOVEL ONDE RESIDIU ANTERIORMENTE. REGISTRA-SE PARA OS DEVIDOS FINS DE DIREITO. ////////////////////</P>",656,95,a comunicante acima qualificada compareceu nessa central de policia de sao bras para informar que na hora data e local acima citados foi vitima de ameaca por parte da nacional jucelena gomes oliveira esta residia ate nov em conjunto com a relatora porem essa mudouse para campinas sp onde esta ate o presente momento tendo expectativa de retorno para belem pa nos proximos dias ao registro deste bop a acusada realiza constantes ofensas e ameacas a integridade fisica da mesma e de seus familiares ao reivindicar posse do imovel onde residiu anteriormente registrase para os devidos fins de direito,"[a, comunicante, acima, qualificada, compareceu, nessa, central, de, policia, de, sao, bras, para, informar, que, na, hora, data, e, local, acima, citados, foi, vitima, de, ameaca, por, parte, da, nacional, jucelena, gomes, oliveira, esta, residia, ate, nov, em, conjunto, com, a, relatora, porem, essa, mudouse, para, campinas, sp, onde, esta, ate, o, presente, momento, tendo, expectativa, de, retorno, para, belem, pa, nos, proximos, dias, ao, registro, deste, bop, a, acusada, realiza, constantes, ofensas, e, ameacas, a, integridade, fisica, da, mesma, e, de, seus, familiares, ao, reivindicar, posse, do, imovel, onde, residiu, anteriormente, registrase, para, os, devidos, fins, de, direito]","[comunicante, qualificada, compareceu, nessa, central, policia, sao, bras, informar, hora, local, citados, vitima, ameaca, parte, nacional, jucelena, gomes, oliveira, residia, ate, nov, conjunto, relatora, porem, mudouse, campinas, sp, onde, ate, presente, momento, tendo, expectativa, retorno, belem, proximos, dias, registro, deste, bop, acusada, realiza, constantes, ofensas, ameacas, integridade, fisica, mesma, familiares, reivindicar, posse, imovel, onde, residiu, anteriormente, registrase]","[comunic, qualific, comparec, ness, centr, polic, sao, bra, inform, hor, local, cit, vit, ameac, part, nacion, jucelen, gom, oliv, resid, ate, nov, conjunt, rela, por, mudous, campin, sp, ond, ate, pres, moment, tend, expect, retorn, bel, prox, dia, registr, dest, bop, acus, realiz, const, ofens, ameac, integr, fisic, mesm, famili, reivindic, poss, imovel, ond, resid, anteri, registras]","[ , o, comunicante, acima, qualificar, comparecer, em esse, central, de, policia, de, sao, bra, para, informar, que, em o, hora, data, e, local, acima, citar, ser, vitimo, de, ameaca, por, parte, de o, nacional, jucelena, Gomes, oliveirar, este, residir, ate, nov, , em, conjunto, com, o, relatora, por, esse, mudouse, para, campina, sp, onde, este, ate, o, presente, momento, ter, expectativa, de, retorno, para, belar, pa, em o, proximo, dia, a o, registro, de este, bop, o, acusar, realizar, constante, ofensa, e, ameaco, o, integridade, fisico, de o, mesmo, e, de, seu, familiar, a o, reivindicar, posse, de o, imovel, onde, residir, anteriormente, registraser, para, o, devido, fim, de, ...]"
1,AMEACA,"<p>//A comunicante acima qualificada compareceu a esta Seccional Urbana para informar que no dia 21/06/2020, por volta das 12:05 estava fazendo seu alimento no fogo a lenha pos avia terminado seu&nbsp;gás, quando sua vizinha a Sra de suposto Nominal RAIMUNDA residente na PASSAGEM; CABO LEAO E/ R

top palavras mais frequentes


In [39]:
def top_freq(lista_de_tokens, k=10):
    lista_unidas = chain.from_iterable(lista_de_tokens)
    quantidade = Counter(lista_unidas)
    return pd.DataFrame(quantidade.most_common(k), columns= ['Tokens', 'Frequência'])

top_freq(df_crimes_limpo['Sem stopwords'], k=15)

,Tokens,Frequência
0,relatora,1095
1,nao,1017
2,declarante,605
3,casa,566
4,dia,394
5,anos,372
6,vitima,330
7,nacional,278
8,registrase,255
9,filho,254


In [ ]:
df_crimes_limpo.to_csv('02_dados_pre_processados.csv', index = False)